In [6]:
import jsonlines
import os
import numpy as np
import pandas as pd
from utils import read_data, extract_data, partition
from evaluation import calculate_score, process_CTC

In [7]:
task_list=('CMeIE', 'CHIP-CDN', 'CHIP-CDEE', 'CHIP-MDCFNPC',
           'CHIP-CTC', 'KUAKE-QIC',
           'IMCS-V2-MRG', 'MedDG',)
pred_path = "pred"
true_path = "true"

In [8]:
target_list = ["test_predictions.json"]

In [9]:
score_dict = {target: [] for target in target_list}
score_dict

{'test_predictions.json': []}

In [5]:
score_dict = {target: [] for target in target_list}
label_dict = {}

for target in target_list:
    target_path = os.path.join(pred_path, target)   #pred/task_predictions.json
    
    if not os.path.exists(os.path.join(pred_path, task_list[0])):  # needs partition
        # all_data = read_data(os.path.join(os.path.join(pred_path, "test_predictions.json"))
        all_data = read_data(target_path)
        partition(extract_data(all_data), task_list, pred_path)
    
    for task in task_list:

        pp = os.path.join(target_path, task)
        tp = os.path.join(true_path, task)
        pp = os.path.join("pred", "test_predictions.json")      #pred_path
        tp = os.path.join("true", "test.json")       #truth_path

        if task == "CHIP-CTC":  # CTC needs post process
            post_process_function = process_CTC
        else:
            post_process_function = None

        score, labels, _ = calculate_score(task, pp, tp, post_process_function)
        score_dict[target].append(score)
        label_dict[task] = labels

100%|██████████| 600/600 [00:00<00:00, 2469658.88it/s]


ZeroDivisionError: division by zero

In [8]:
import tqdm
# 假设 read_data, save_data, extract_data, partition 函数已经定义
# 请确保这些函数的实现与您之前提供的代码一致

# 假设 pred_path 和 task_list 已经被定义
# pred_path = '/path/to/predictions'  # 预测结果的根目录路径
# task_list = ['task1', 'task2', 'task3']  # 任务列表

# 定义 read_data 函数（示例）
def read_data(data_path):
    with jsonlines.open(data_path, "r") as f:
        data = [meta_data for meta_data in f]
    return data

# 定义 save_data 函数（示例）
def save_data(data_path, data):
    with jsonlines.open(data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

# 定义 extract_data 函数（示例）
def extract_data(data):

    data_dict = {}

    for meta_data in data:
        if meta_data['task_dataset'] not in data_dict.keys():
            data_dict[meta_data['task_dataset']] = []
        data_dict[meta_data['task_dataset']].append(meta_data)
    print("extract conpletion")

    return data_dict

# 定义 partition 函数（示例）
def partition(data_dict, task_list, output_path):
    for task in task_list:
        task_path = os.path.join(output_path, task)
        if not os.path.exists(task_path):
            os.makedirs(task_path)

        save_data(os.path.join(task_path, "lhy/MOELoRA-peft/datasets/test.json"), data_dict[task])

# 主脚本逻辑
# 读取预测结果
target_path = "test.json"  # 预测结果文件路径
all_data = read_data(target_path)

# 提取和组织数据
extracted_data = extract_data(all_data)

# 分区并保存数据
partition(extracted_data, task_list, true_path)


extract conpletion


FileNotFoundError: [Errno 2] No such file or directory: 'true/CMeIE/lhy/MOELoRA-peft/datasets/test.json'

In [72]:
res_data, res_key = [], []
for key, value in score_dict.items():
    res_data.append(value)
    res_key.append(key)

res_df = pd.DataFrame(columns=task_list,
                      index=res_key,
                      data=res_data)
res_df["average"] = res_df.mean(axis=1)

In [73]:
res_df.head(20)

,CMeIE,CHIP-CDN,CHIP-CDEE,CHIP-MDCFNPC,CHIP-CTC,KUAKE-QIC,IMCS-V2-MRG,MedDG,average
test_predictions.json,0.432153,0.881279,0.532281,0.786336,0.886364,0.865404,0.376379,0.113989,0.609273


In [74]:
try:
    new_res_df = res_df.drop(columns=["CHIP-STS", "KUAKE-IR", "average"])
except:
    new_res_df = res_df
new_res_df["average"] = new_res_df.mean(axis=1)
new_res_df.head(50)

,CMeIE,CHIP-CDN,CHIP-CDEE,CHIP-MDCFNPC,CHIP-CTC,KUAKE-QIC,IMCS-V2-MRG,MedDG,average
test_predictions.json,0.432153,0.881279,0.532281,0.786336,0.886364,0.865404,0.376379,0.113989,0.609273


In [75]:
for st in ["CHIP-CDN", "CHIP-MDCFNPC", "IMCS-V2-MRG", "KUAKE-QIC"]: 
    score, _, _ = calculate_score(st, 
                                  "pred/%s/test_predictions.json" % st, 
                                  "true/%s/test.json" %st, 
                                  post_process_function)
    print("The score for task %s is: %.5f" % (st, score))

100%|██████████| 600/600 [00:00<00:00, 646603.91it/s]


The score for task CHIP-CDN is: 0.88128


100%|██████████| 600/600 [00:00<00:00, 138868.91it/s]


The score for task CHIP-MDCFNPC is: 0.78634
The score for task IMCS-V2-MRG is: 0.37638
The score for task KUAKE-QIC is: 0.86540
